# Data Prep for Crowd Annotation Pipeline

1. Collect raw data 
2. Adjust contrast of images
3. Chop up images into manageable pieces
4. Make into montages
5. Upload to Figure8

Files are named by these scripts such that the code blocks can run back-to-back with minimal input. For this reason, it is recommended that users run through the whole pipeline before processing another set of images.

In [4]:
# import statements
from __future__ import absolute_import

from io import BytesIO

from IPython.display import Image
from ipywidgets import interact, interactive, fixed
import matplotlib as mpl
from skimage import data, filters, io, img_as_uint
import numpy as np
import skimage as sk
import os
from scipy import ndimage
import scipy
import sys
from ipywidgets import interact
from skimage.io import imread
import matplotlib.pyplot as plt

import scipy.ndimage as ndi

%matplotlib inline

from dcde.pre_annotation.montage_makers import montage_maker, multiple_montage_maker
from dcde.pre_annotation.overlapping_chopper import overlapping_crop_dir
from dcde.pre_annotation.aws_upload import aws_upload, upload
from dcde.pre_annotation.montage_to_csv import csv_maker
from dcde.pre_annotation.fig_eight_upload import fig_eight
from dcde.pre_annotation.contrast_adjustment import contrast

In [5]:
#sometimes raw images are in .tif stacks, not individual .tif files
#optional code block for turning into individual slices

## 2. Adjust contrast of images
Before doing anything else, we need to adjust the contrast of the raw data. contrast_adjustment blurs the data using a gaussian filter, finds the edges, inverts, and does additional equalization if needed. The user defines the parameters needed using the widgets below.

In [6]:
# Define path to desired raw directory
base_dir = "test/pics"
raw_folder = "pics181220"
identifier = "test"

dirpath = os.path.join(base_dir, raw_folder + "/raw")


In [7]:
# Functions that the notebook will play with interactively to run widgets
def arr2img(arr):
    """Display a 2- or 3-d numpy array as an image."""
    if arr.ndim == 2:
        format, cmap = 'png', mpl.cm.gray
    elif arr.ndim == 3:
        format, cmap = 'jpg', None
    else:
        raise ValueError("Only 2- or 3-d arrays can be displayed as images.")
    # Don't let matplotlib autoscale the color range so we can control overall luminosity
    vmax = 255 if arr.dtype == 'uint8' else 1.0
    with BytesIO() as buffer:
        mpl.image.imsave(buffer, arr, format=format, cmap=cmap)
        out = buffer.getvalue()
    return Image(out)

def choose_img(name):
    """Used to choose which image we want to use for the widget tester"""
    global img
    filepath = os.path.join(dirpath, name)
    img = img_as_uint(imread(filepath))
    return arr2img(img)

def edit_image(image, sigma=1.0, equalize_hist=False, equalize_adapthist=False):
    """Used to edit the image using the widget tester"""
    global hist
    global adapthist
    global gaussian_sigma
    
    new_image = filters.gaussian(image, sigma=sigma, multichannel=False)
    new_image += 1000*sk.filters.sobel(new_image)
    new_image[:] = -1.0*new_image[:]
    new_image=sk.exposure.rescale_intensity(new_image, in_range = 'image', out_range = 'float')
    
    if(equalize_hist == True):
        #new_image=sk.exposure.rescale_intensity(new_image, in_range = 'image', out_range = 'np.uint16')
        new_image = sk.exposure.equalize_hist(new_image, nbins=256, mask=None)
        
    if(equalize_adapthist == True):
        new_image = sk.exposure.equalize_adapthist(new_image, kernel_size=None, clip_limit=0.01, nbins=256)
     
    
    hist = equalize_hist
    adapthist = equalize_adapthist
    gaussian_sigma = sigma
        
    
    return arr2img(new_image)



In [8]:
# Choose which raw image you would like to use to test on the contrast adjustment
interact(choose_img, name=sorted(set(os.listdir(dirpath))));
    

interactive(children=(Dropdown(description='name', options=('181220_Hyb1_pos1_4umSteps-0001.tif', '181220_Hyb1…

In [9]:
# Test with choosen image to fix adjustment parameters
interact(edit_image, image=fixed(img), sigma=(0.0,4,0.3));

interactive(children=(FloatSlider(value=1.0, description='sigma', max=4.0, step=0.3), Checkbox(value=False, de…

In [10]:
# With choosen parameters, process all the raw data in the folder
contrast(base_dir, raw_folder, identifier, gaussian_sigma, hist, adapthist)

Processed data will be located at test/pics/pics181220_contrast_adjusted


## 3. Chop up images into manageable pieces

Each full-size image usually has many cells in it. This makes them difficult to fully annotate! For ease of annotation (and better results), each frame is chopped up into smaller, overlapping frames, ultimately creating a set of movies. 

These smaller movies can be made with overlapping edges, making it easier to stitch annotations together into one large annotated movie (in the post-annotation pipeline). A large overlap will result in redundant annotations.

Even if you want to process the full-sized image, run the chopper with num_segments of 1. The montage makers are written to run on the output of the chopper.

In [ ]:
# base_direc = "/home/geneva/Desktop/Nb_testing/"
# raw_direc = os.path.join(base_direc, "MouseBrain_s7_nuclear")
# identifier = "MouseBrain_s7_nuc"

num_x_segments = 5
num_y_segments = 5
overlap_perc = 10

In [ ]:
raw_direc = "test/pics/pics181220/processed"
overlapping_crop_dir(raw_direc, identifier, num_x_segments, num_y_segments, overlap_perc)

## 4. Make into montages
multiple_montage_maker is written to run on the output of the chopper, ie the folder where each chopped movie folder is saved. It will make montages of each subfolder according to the variables specified. It will make more than one montage per subfolder if there are enough frames to do so.

The variables used in multiple_montage_maker are saved in a JSON file so they can be reused in post-annotation processing.

In [ ]:
montage_len = 15

direc = raw_direc + "_chopped_" + str(num_x_segments) + "_" + str(num_y_segments)
#direc = "/home/geneva/Desktop/Nb_testing/nuclear_test_chopped_4_4"

save_direc = os.path.join(base_direc, identifier + "_montages_" + str(num_x_segments) + "_" + str(num_y_segments))
#save_direc = "/home/geneva/Desktop/Nb_testing/montages"

log_direc = os.path.join(base_direc, "json_logs")

row_length = 5
x_buffer = 5
y_buffer = 5

In [ ]:
multiple_montage_maker(montage_len, direc, save_direc, identifier, 
                       num_x_segments, num_y_segments, row_length, x_buffer, y_buffer, log_direc)

## 5. Upload to Figure Eight
Now that the images are processed into montages, they need to be uploaded to an AWS bucket and submitted to Figure Eight. This involves uploading the files to AWS, making a CSV file with the links to the uploaded images, and using that CSV file to create a Figure Eight job.

### Upload files to AWS
aws_upload will look for image files in the specified directory (folder_to_upload, set by default to be wherever the output of multiple_montage_maker was saved) and upload them into a bucket.

For the Van Valen lab, the default bucket is "figure-eight-deepcell" and keys (aws_folder + file names) correspond to the file structure of our data server.

aws_upload returns a list of the urls to which images were uploaded.

In [ ]:
#import os

bucket_name = "figure-eight-deepcell" #default
aws_folder = "MouseBrain/set7"
folder_to_upload = save_direc #usually .../montages
#data_to_upload = "/home/geneva/Desktop/Nb_testing/montages/"

uploaded_montages = aws_upload(bucket_name, aws_folder, folder_to_upload)

#os.path.join("https://s3.us-east-2.amazonaws.com", bucket_name, aws_folder)
#print(uploaded_montages)
#from io_utils import get_img_names
#imgs_to_upload = get_img_names(folder_to_upload)
#for index, img in enumerate(imgs_to_upload):
#    print(img)
#    print(os.path.join(folder_to_upload, img))

### Make CSV file
Figure Eight jobs can be created easily by using a CSV file where each row contains information about one task. For our jobs, each row has the link to the location of one montage, and information about that montage (currently, just the "identifier" specified at the beginning of the pipeline). The CSV file is saved as "identifier".csv in a folder that only holds CSVs. CSV folders are usually in cell-type directories, so identifiers should be able to distinguish between sets, parts, etc.

In [ ]:
#identifier = "test"
csv_direc = os.path.join(base_direc, "CSV")

In [ ]:
csv_maker(uploaded_montages, identifier, csv_direc)

### Create Figure Eight job
The Figure Eight API allows us to create a new job and upload data to it from this notebook. However, since our jobs don't include required test questions, editing job information such as the title of the job must be done via the website. This section of the notebook uses the API to create a job and upload data to it, then reminds the user to finish editing the job on the website.

Some sample job IDs to copy are provided below.

In [ ]:
#job_id_to_copy = 1344258 #Elowitz timelapse RFP pilot
job_id_to_copy = 1346216 #Deepcell MouseBrain 3x5
#job_id_to_copy = 1306431 #Deepcell overlapping Mibi
#job_id_to_copy = 1292179 #Deepcell HEK
#job_id_to_copy =

In [ ]:
from dcde.pre_annotation.fig_eight_upload import fig_eight

fig_eight(csv_direc, identifier, job_id_to_copy)